In [1]:
from huggingface_hub import login, notebook_login
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, BitsAndBytesConfig, AutoConfig
import torch
from textwrap import fill
from langchain.prompts import PromptTemplate
import locale
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores.utils import filter_complex_metadata
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from huggingface_hub import login
import os
login(os.getenv('HUGGINGFACE_TOKEN'))

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from concurrent.futures import ProcessPoolExecutor, as_completed
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Get all .pdf files in the pdfs/ directory and subdirectories
pdf_files = []
for root, dirs, files in os.walk('pdfs'):
    for file in files:
        if file.endswith('.pdf'):
            pdf_files.append(os.path.join(root, file))

# Create loaders for each .pdf file
loaders = [PyMuPDFLoader(fn) for fn in pdf_files]

def process_loader(loader):
    md_doc = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
    documents = text_splitter.split_documents(md_doc)
    return documents

chunked_pdf_doc = []

print("Loading documents")
with ProcessPoolExecutor(max_workers=10) as executor:  # Reduce workers to avoid memory issues
    futures = [executor.submit(process_loader, loader) for loader in loaders]
    for future in as_completed(futures):
        logging.info(f"Process {future} completed")
        chunked_pdf_doc.extend(future.result())

print(len(chunked_pdf_doc))

Loading documents


2025-03-03 16:52:48,326 - INFO - Process <Future at 0x7fdcba29e9e0 state=finished returned list> completed
2025-03-03 16:52:48,339 - INFO - Process <Future at 0x7fdcba29d6c0 state=finished returned list> completed
2025-03-03 16:52:48,345 - INFO - Process <Future at 0x7fdcba29e920 state=finished returned list> completed
2025-03-03 16:52:48,369 - INFO - Process <Future at 0x7fdcba29e620 state=finished returned list> completed
2025-03-03 16:52:48,398 - INFO - Process <Future at 0x7fdcba29e860 state=finished returned list> completed


93


In [4]:
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs,
    multi_process=True,
)

vector_store = FAISS.from_documents(chunked_pdf_doc, embeddings)

/tmp/ipykernel_29252/920886965.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
2025-03-03 16:52:48,530 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2025-03-03 16:52:53,379 - INFO - Start multi-process pool on devices: cuda:0


Chunks:   0%|          | 0/10 [00:00<?, ?it/s]

2025-03-03 16:53:02,428 - INFO - Loading faiss with AVX2 support.
2025-03-03 16:53:02,429 - INFO - Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2025-03-03 16:53:02,430 - INFO - Loading faiss.
2025-03-03 16:53:02,464 - INFO - Successfully loaded faiss.


In [5]:
# Specify the directory where you want to save the FAISS index and metadata
save_directory = "faiss_index"

# Save the FAISS index and metadata to the specified directory
vector_store.save_local(save_directory)

In [6]:
model_name = "meta-llama/Llama-3.2-1B-Instruct" 

quantization_config = BitsAndBytesConfig()

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="cuda",
                                             quantization_config=quantization_config,
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

gen_cfg = GenerationConfig.from_pretrained(model_name)
gen_cfg.max_new_tokens=512
gen_cfg.temperature=0.0000001 
gen_cfg.return_full_text=True
gen_cfg.do_sample=True
gen_cfg.repetition_penalty=1.11

pipe=pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=gen_cfg
)

llm = HuggingFacePipeline(pipeline=pipe)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda


In [7]:
prompt_template_llama3 = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a resume reviewer. You will be given a resume. Use the following context to answer the question and give tailored feedback
on how to improve the resume. Be concise and to the point. Return in this format:

<general review of the resume>
[The text of the general review]
</general review of the resume>

<tailored feedback answering the question>
[The text of the tailored feedback]
</tailored feedback>

<Next steps for improvement>
[The text of the next steps for improvement]
</Next steps for improvement>

{context}<|eot_id|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt_template=prompt_template_llama3

prompt = PromptTemplate(
    input_variables=["text"],
    template=prompt_template,
)
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [8]:
Chain_pdf = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={'k': 10, 'score_threshold': 0.2}),
    chain_type_kwargs={"prompt": prompt},
)

query = """Does the resume highlight leadership experience effectively?

Work Experience:
Software Engineering Intern | Google | Summer 2024
- Developed and optimized backend services for a large-scale distributed system, improving performance by 20%.
- Led a team of three interns in implementing a feature that reduced data retrieval time by 30%.

Teaching Assistant | Amherst College | Fall 2023 - Present
- Led weekly discussions and mentored students in algorithms and data structures.
- Created practice problems that improved students’ understanding of computational complexity.
"""
result = Chain_pdf.invoke(query)
print(fill(result['result'].strip(), width=200))
print("##########################################################################")

query = """Are the technical skills presented in a way that aligns with industry expectations?

Work Experience:
Software Engineer | Startup XYZ | 2023 - Present
- Built RESTful APIs using Django, handling over 100k requests daily.
- Developed a CI/CD pipeline that reduced deployment time by 50%.

Research Assistant | AI Lab | 2022 - 2023
- Implemented neural network verification techniques in Python.
- Optimized matrix computations, reducing runtime by 40%.
"""
result = Chain_pdf.invoke(query)
print(fill(result['result'].strip(), width=200))

2025-03-03 16:56:41,962 - INFO - Start multi-process pool on devices: cuda:0


Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/liam/anaconda3/envs/vectordb/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
2025-03-03 16:56:56,624 - INFO - Start multi-process pool on devices: cuda:0


<|begin_of_text|><|start_header_id|>system<|end_header_id|>  You are a resume reviewer. You will be given a resume. Use the following context to answer the question and give tailored feedback on how
to improve the resume. Be concise and to the point. Return in this format:  <general review of the resume> [The text of the general review] </general review of the resume>  <tailored feedback
answering the question> [The text of the tailored feedback] </tailored feedback>  <Next steps for improvement> [The text of the next steps for improvement] </Next steps for improvement>  describe
entire, sometimes multi-year experiences into a very short amount of space. Each position requires 2-4 action-oriented bullet points describing not only what you did, but how you did it, emphasizing
data and the impact of your work. ● List the organization, city and state, your role, and the month(s) and year(s) you were/are involved Loeb Center for Career Exploration and Planning |
amherst.edu/campuslife/care

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>  You are a resume reviewer. You will be given a resume. Use the following context to answer the question and give tailored feedback on how
to improve the resume. Be concise and to the point. Return in this format:  <general review of the resume> [The text of the general review] </general review of the resume>  <tailored feedback
answering the question> [The text of the tailored feedback] </tailored feedback>  <Next steps for improvement> [The text of the next steps for improvement] </Next steps for improvement>  Skills &
Interests Language Skills: Fluent in Spanish; Advanced in French Digital Tools: Google Suite, Microsoft Office Suite, Zoom, Slack, Zotero Interests: Baking pastries, rock climbing, political podcasts,
fiction writing Additional Categories These are all optional categories that may be relevant depending on your circumstance, field of study or intended industry. Selected Projects ○ This is a section
most often u